# Additional End of week Exercise - week 2

Now use everything you've learned from Week 2 to build a full prototype for the technical question/answerer you built in Week 1 Exercise.

This should include a Gradio UI, streaming, use of the system prompt to add expertise, and the ability to switch between models. Bonus points if you can demonstrate use of a tool!

If you feel bold, see if you can add audio input so you can talk to it, and have it respond with audio. ChatGPT or Claude can help you, or email me if you have questions.

I will publish a full solution here soon - unless someone beats me to it...

There are so many commercial applications for this, from a language tutor, to a company onboarding solution, to a companion AI to a course (like this one!) I can't wait to see your results.

In [11]:
# imports

import os
import json
from dotenv import load_dotenv
from openai import OpenAI
import gradio as gr

In [12]:
# Initialization

load_dotenv()

openai_api_key = os.getenv('OPENAI_API_KEY')
if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")
    
MODEL = "gpt-4o-mini"
openai = OpenAI()

OpenAI API Key exists and begins sk-proj-


In [13]:
system_message = "You are a helpful assistant for an Airline called FlightAI."
system_message += "Give short, courteous answers, no more than 1 sentence. "
system_message += "Always be accurate. If you don't know the answer, say so."

In [14]:
# Let's start by making a useful function

ticket_prices = {"new york": "$999", "london": "$799", "paris": "$899", "tokyo": "$1400", "berlin": "$499", "new delhi": "$1111"}

def get_ticket_price(destination_city):
    print(f"Tool get_ticket_price called for {destination_city}")
    city = destination_city.lower()
    return ticket_prices.get(city, "Unknown")

In [15]:
# There's a particular dictionary structure that's required to describe our function:

price_function = {
    "name": "get_ticket_price",
    "description": "Get the price of a return ticket to the destination city. Call this whenever you need to know the ticket price, for example when a customer asks 'How much is a ticket to this city'",
    "parameters": {
        "type": "object",
        "properties": {
            "destination_city": {
                "type": "string",
                "description": "The city that the customer wants to travel to",
            },
        },
        "required": ["destination_city"],
        "additionalProperties": False
    }
}

In [16]:
#Assignment -- adding one more tool to book a flight

flight_booking_status = {"new york": "blocked", "london": "booked", "paris": "can't be booked", "tokyo": "pending", "berlin": "payment issue", "new delhi": "cancelled" }

#this is a tool function that will be used by a GPT model

def get_flight_booking_status(destination_city):
    print(f"Tool get_flight_booking_status for {destination_city}")
    city = destination_city.lower()
    return flight_booking_status.get(city, "Unknown")
    

In [17]:
# Describe metadata about the above tool function so that model can understand it

flight_booking_status_function = {
    "name": "get_flight_booking_status",
    "description": "Book a flight and get its booking status. Show your answer in polite way, like flight booked, cancelled, pending or can't be booked etc...'",
    "parameters": {
        "type": "object",
        "properties": {
            "destination_city": {
                "type": "string",
                "description": "The city that the customer wants to book a flight for",
            },
        },
        "required": ["destination_city"],
        "additionalProperties": False
    }
}

In [18]:
# Some imports for handling images

import base64
from io import BytesIO
from PIL import Image

In [19]:
def artist(city):
    image_response = openai.images.generate(
            model="dall-e-3",
            prompt=f"An image representing a vacation in {city}, showing tourist spots and everything unique about {city}, in a vibrant pop-art style",
            size="1024x1024",
            n=1,
            response_format="b64_json",
        )
    image_base64 = image_response.data[0].b64_json
    image_data = base64.b64decode(image_base64)
    return Image.open(BytesIO(image_data))

In [20]:
from pydub import AudioSegment
from pydub.playback import play

def talker(message):
    response = openai.audio.speech.create(
      model="tts-1",
      voice="alloy",    # Also, try replacing onyx with alloy
      input=message
    )
    
    audio_stream = BytesIO(response.content)
    audio = AudioSegment.from_file(audio_stream, format="mp3")
    play(audio)

In [21]:
#2 tools - add the above booking status function to the tools list
tools = [{"type": "function", "function": price_function}, {"type": "function", "function": flight_booking_status_function}]

In [22]:
# We have to write a new function handle_multi_tool_calls to handle both the tools:

def handle_multi_tool_calls(message):
    tool_call = message.tool_calls[0]
    arguments = json.loads(tool_call.function.arguments)
    city = arguments.get('destination_city')
    price = get_ticket_price(city)
    booking_status = get_flight_booking_status(city)
    response = {
        "role": "tool",
        "content": json.dumps({"destination_city": city,"price": price, "booking_status": booking_status}),
        "tool_call_id": message.tool_calls[0].id
    }
    return response, city

In [23]:
#update the chat function to use both the tools
def chat(history):
    messages = [{"role": "system", "content": system_message}] + history
    response = openai.chat.completions.create(model=MODEL, messages=messages, tools=tools)
    image = None
    
    if response.choices[0].finish_reason=="tool_calls":
        message = response.choices[0].message
        response, city = handle_multi_tool_calls(message)
        image = city
        messages.append(message)
        messages.append(response)
        image = artist(city)
        response = openai.chat.completions.create(model=MODEL, messages=messages)
        
    reply = response.choices[0].message.content
    history += [{"role":"assistant", "content":reply}]

    # Comment out or delete the next line if you'd rather skip Audio for now..
    talker(reply)
    
    return history, image

In [24]:
# More involved Gradio code as we're not using the preset Chat interface!
# Passing in inbrowser=True in the last line will cause a Gradio window to pop up immediately.

with gr.Blocks() as ui:
    with gr.Row():
        chatbot = gr.Chatbot(height=500, type="messages")
        image_output = gr.Image(height=500)
    with gr.Row():
        entry = gr.Textbox(label="Chat with our AI Assistant:")
    with gr.Row():
        clear = gr.Button("Clear")

    def do_entry(message, history):
        history += [{"role":"user", "content":message}]
        return "", history

    entry.submit(do_entry, inputs=[entry, chatbot], outputs=[entry, chatbot]).then(
        chat, inputs=chatbot, outputs=[chatbot, image_output]
    )
    clear.click(lambda: None, inputs=None, outputs=chatbot, queue=False)

ui.launch(inbrowser=True)

* Running on local URL:  http://127.0.0.1:7868

To create a public link, set `share=True` in `launch()`.


Input #0, wav, from '/var/folders/w_/f1q2tn493fng_1vt_6bpj99w0000gp/T/tmpbv4ah2zq.wav':
  Duration: 00:00:02.02, bitrate: 384 kb/s
  Stream #0:0: Audio: pcm_s16le ([1][0][0][0] / 0x0001), 24000 Hz, 1 channels, s16, 384 kb/s
   1.94 M-A:  0.000 fd=   0 aq=    0KB vq=    0KB sq=    0B    0.01 M-A:  0.000 fd=   0 aq=   54KB vq=    0KB sq=    0B    0.21 M-A:  0.000 fd=   0 aq=   46KB vq=    0KB sq=    0B    0.44 M-A:  0.000 fd=   0 aq=   34KB vq=    0KB sq=    0B    0.64 M-A:  0.000 fd=   0 aq=   22KB vq=    0KB sq=    0B    0.85 M-A: -0.000 fd=   0 aq=   14KB vq=    0KB sq=    0B    1.08 M-A:  0.000 fd=   0 aq=    2KB vq=    0KB sq=    0B    1.31 M-A:  0.000 fd=   0 aq=    0KB vq=    0KB sq=    0B    1.54 M-A:  0.000 fd=   0 aq=    0KB vq=    0KB sq=    0B    1.77 M-A: -0.000 fd=   0 aq=    0KB vq=    0KB sq=    0B 


Tool get_ticket_price called for New Delhi
Tool get_flight_booking_status for New Delhi


Input #0, wav, from '/var/folders/w_/f1q2tn493fng_1vt_6bpj99w0000gp/T/tmpjuhxs96g.wav':
  Duration: 00:00:07.46, bitrate: 384 kb/s
  Stream #0:0: Audio: pcm_s16le ([1][0][0][0] / 0x0001), 24000 Hz, 1 channels, s16, 384 kb/s
   7.37 M-A:  0.000 fd=   0 aq=    0KB vq=    0KB sq=    0B    0.04 M-A:  0.000 fd=   0 aq=  100KB vq=    0KB sq=    0B    0.27 M-A:  0.000 fd=   0 aq=  104KB vq=    0KB sq=    0B    0.50 M-A:  0.000 fd=   0 aq=  104KB vq=    0KB sq=    0B    0.73 M-A:  0.000 fd=   0 aq=  100KB vq=    0KB sq=    0B    0.96 M-A:  0.000 fd=   0 aq=  104KB vq=    0KB sq=    0B    1.19 M-A:  0.000 fd=   0 aq=  104KB vq=    0KB sq=    0B    1.39 M-A:  0.000 fd=   0 aq=  104KB vq=    0KB sq=    0B    1.62 M-A:  0.000 fd=   0 aq=  104KB vq=    0KB sq=    0B    1.85 M-A:  0.000 fd=   0 aq=  104KB vq=    0KB sq=    0B    2.08 M-A:  0.000 fd=   0 aq=  104KB vq=    0KB sq=    0B    2.31 M-A:  0.000 fd=   0 aq=  104KB vq=    0KB sq=    0B    2.54 M-A: -0.000 fd=   0 aq=  104KB vq=    0KB sq=   


Tool get_ticket_price called for san francisco
Tool get_flight_booking_status for san francisco


Input #0, wav, from '/var/folders/w_/f1q2tn493fng_1vt_6bpj99w0000gp/T/tmpfn_i3dnq.wav':
  Duration: 00:00:06.53, bitrate: 384 kb/s
  Stream #0:0: Audio: pcm_s16le ([1][0][0][0] / 0x0001), 24000 Hz, 1 channels, s16, 384 kb/s
   6.42 M-A: -0.000 fd=   0 aq=    0KB vq=    0KB sq=    0B    0.04 M-A:  0.000 fd=   0 aq=  104KB vq=    0KB sq=    0B    0.27 M-A: -0.000 fd=   0 aq=  104KB vq=    0KB sq=    0B    0.50 M-A:  0.000 fd=   0 aq=  104KB vq=    0KB sq=    0B    0.73 M-A:  0.000 fd=   0 aq=  100KB vq=    0KB sq=    0B    0.96 M-A:  0.000 fd=   0 aq=  104KB vq=    0KB sq=    0B    1.19 M-A:  0.000 fd=   0 aq=  104KB vq=    0KB sq=    0B    1.39 M-A:  0.000 fd=   0 aq=  104KB vq=    0KB sq=    0B    1.62 M-A:  0.000 fd=   0 aq=  104KB vq=    0KB sq=    0B    1.85 M-A: -0.000 fd=   0 aq=  104KB vq=    0KB sq=    0B    2.08 M-A:  0.000 fd=   0 aq=  104KB vq=    0KB sq=    0B    2.30 M-A: -0.000 fd=   0 aq=  104KB vq=    0KB sq=    0B    2.53 M-A:  0.000 fd=   0 aq=  104KB vq=    0KB sq=   


Tool get_ticket_price called for New York
Tool get_flight_booking_status for New York


Input #0, wav, from '/var/folders/w_/f1q2tn493fng_1vt_6bpj99w0000gp/T/tmp9dgx9p9j.wav':
  Duration: 00:00:06.00, bitrate: 384 kb/s
  Stream #0:0: Audio: pcm_s16le ([1][0][0][0] / 0x0001), 24000 Hz, 1 channels, s16, 384 kb/s
   5.93 M-A:  0.000 fd=   0 aq=    0KB vq=    0KB sq=    0B    0.00 M-A:  0.000 fd=   0 aq=  104KB vq=    0KB sq=    0B    0.23 M-A:  0.000 fd=   0 aq=  104KB vq=    0KB sq=    0B    0.46 M-A:  0.000 fd=   0 aq=  104KB vq=    0KB sq=    0B    0.69 M-A:  0.000 fd=   0 aq=  104KB vq=    0KB sq=    0B    0.92 M-A:  0.000 fd=   0 aq=  104KB vq=    0KB sq=    0B    1.14 M-A:  0.000 fd=   0 aq=  104KB vq=    0KB sq=    0B    1.37 M-A:  0.000 fd=   0 aq=  104KB vq=    0KB sq=    0B    1.60 M-A:  0.000 fd=   0 aq=  104KB vq=    0KB sq=    0B    1.83 M-A:  0.000 fd=   0 aq=  104KB vq=    0KB sq=    0B    2.06 M-A:  0.000 fd=   0 aq=  104KB vq=    0KB sq=    0B    2.29 M-A:  0.000 fd=   0 aq=  104KB vq=    0KB sq=    0B    2.52 M-A:  0.000 fd=   0 aq=  104KB vq=    0KB sq=   


Tool get_ticket_price called for New York
Tool get_flight_booking_status for New York


Input #0, wav, from '/var/folders/w_/f1q2tn493fng_1vt_6bpj99w0000gp/T/tmpyddd2w8h.wav':
  Duration: 00:00:05.30, bitrate: 384 kb/s
  Stream #0:0: Audio: pcm_s16le ([1][0][0][0] / 0x0001), 24000 Hz, 1 channels, s16, 384 kb/s
   5.25 M-A:  0.000 fd=   0 aq=    0KB vq=    0KB sq=    0B    0.01 M-A:  0.000 fd=   0 aq=  104KB vq=    0KB sq=    0B    0.24 M-A:  0.000 fd=   0 aq=  104KB vq=    0KB sq=    0B    0.47 M-A:  0.000 fd=   0 aq=  100KB vq=    0KB sq=    0B    0.70 M-A:  0.000 fd=   0 aq=  104KB vq=    0KB sq=    0B    0.93 M-A:  0.000 fd=   0 aq=  104KB vq=    0KB sq=    0B    1.16 M-A:  0.000 fd=   0 aq=  104KB vq=    0KB sq=    0B    1.39 M-A:  0.000 fd=   0 aq=  104KB vq=    0KB sq=    0B    1.63 M-A:  0.000 fd=   0 aq=  104KB vq=    0KB sq=    0B    1.83 M-A:  0.000 fd=   0 aq=  104KB vq=    0KB sq=    0B    2.03 M-A:  0.000 fd=   0 aq=  104KB vq=    0KB sq=    0B    2.23 M-A:  0.000 fd=   0 aq=  104KB vq=    0KB sq=    0B    2.46 M-A:  0.000 fd=   0 aq=   93KB vq=    0KB sq=   